## NonLocal

[《Non-local Neural Networks》](https://arxiv.org/abs/1711.07971)发表于CVPR2018，用于作为处理动作分类的一种方法。

### 算法原理简介

<div align=center>
<!-- <img src=./src/pic/nonlocal_block.png>  -->
<img src=./pics/nonlocal_block.png> 

Figure 1 nonlocal_block </div>
NonLocal是一个灵活的构建块，可以很容易地与卷积/循环层一起使用。它可以添加到深度神经网络的前面部分，不像fc层经常在最后使用。允许构建一个结合non-local信息和局部信息的更丰富的层次结构。论文中的nonlocal将某一位置的响应当做是一种从特征图谱所有位置的加权和来计算，这些位置既可以代表空间位置, 也可以代表时间, 时空等。Nonlocal其实和self-attention机制十分相关。在文中，为了能够将提出的nonlocal block当作一个组件自由的接入到各个神经网络中，作者设计的nonlocal 操作使得输入输出大小一致，具体实现公式如下：
 
 <div align=center>
<img src=./pics/Formulation.png> 

Formulation</div>

公式中，x代表输入，y代表输出，i和j分别代表输入的某个空间位置，xi是一个向量，维数跟x的channel数一样，f是一个计算任意两点相似关系的函数，g是一个映射函数，将一个点映射成一个向量，即该点的特征。为了计算输出层的一个点，需要将输入的每个点都考虑一遍，考虑的方式就和attention机制类似：过程中mask则是根据f函数给出，再和g映射函数相乘，最后求和，输出的某个点在原图上的attention。每个点以这样的方式计算，最后得到一个nonlocal的“attention map”。

<div align=center>
<img src=./pics/baseline_ResNet50_C2D.png> 

Table 1 baseline_ResNet50_C2D</div>
表1显示了ResNet-50骨干网下的C2D基线。在这个存储库中，我们使用ResNet-50骨干网下的Inflated 3D ConvNet(I3D)。可以通过“inflating”内核将表1中的C2D模型转换为3D卷积模型。例如，一个2D的k×k内核可以被扩展为一个横跨t帧的3D t×k×k内核。我们添加5个块(3到res4, 2到res3，每隔一个残差块)。欲了解更多细节信息，请阅读论文[《Non-local Neural Networks》](https://arxiv.org/abs/1711.07971)。


### 环境准备
```text
git clone https://gitee.com/yanlq46462828/zjut_mindvideo.git
cd zjut_mindvideo

# Please first install mindspore according to instructions on the official website: https://www.mindspore.cn/install

pip install -r requirements.txt
pip install -e .
```
### 训练流程


In [1]:
from mindspore import nn
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor
from mindspore.nn.loss import SoftmaxCrossEntropyWithLogits
from mindspore.nn.metrics import Accuracy

from mindvideo.utils.check_param import Validator,Rel

##### 数据集加载

将数据集下载到以下路径，或者根据自己需求改变路径即可。数据集下载链接：https://deepmind.com/research/open-source/kinetics
通过基于VideoDataset编写的Kinetic400类来加载kinetic400数据集。

In [3]:
from mindvideo.data.kinetics400 import Kinetic400
# Data Pipeline.
dataset = Kinetic400(path='/home/publicfile/kinetics-400',
                    split="train",
                    shuffle=True,
                    seq=32,
                    seq_mode='interval',
                    num_parallel_workers=1,
                    batch_size=6,
                    repeat_num=1,
                    frame_interval=6)
ckpt_save_dir = './nonlocal'

/home/publicfile/kinetics-400/cls2index.json


##### 数据处理

用VideoShortEdgeResize根据短边来进行Resize，再用VideoRandomCrop对Resize后的视频进行随机裁剪，再用VideoRandomHorizontalFlip根据概率对视频进行水平翻转，通过VideoRescale对视频进行缩放，利用VideoReOrder对维度进行变换，再用VideoNormalize进行归一化处理。

In [4]:
from mindvideo.data.transforms import VideoRandomCrop, VideoRandomHorizontalFlip, VideoRescale
from mindvideo.data.transforms import VideoNormalize, VideoShortEdgeResize, VideoReOrder
# Data Pipeline.
transforms = [VideoShortEdgeResize(size=256, interpolation='bicubic'),
                VideoRandomCrop([224, 224]),
                VideoRandomHorizontalFlip(0.5),
                VideoRescale(),
                VideoReOrder([3, 0, 1, 2]),
                VideoNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.255])]

dataset.transform = transforms
dataset_train = dataset.run()
Validator.check_int(dataset_train.get_dataset_size(), 0, Rel.GT)
step_size = dataset_train.get_dataset_size()

[WARNING] ME(132676:140166743865152,MainProcess):2023-03-13-07:47:59.541.217 [mindspore/dataset/core/validator_helpers.py:804] 'Compose' from mindspore.dataset.transforms.py_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Compose' from mindspore.dataset.transforms instead.


##### 网络构建
1. Nonlocal最重要的结构是NonlocalBlockND（nn.Cell），该block包含四种成对相似度计算公式，以dot_product为例，主要通过三个Conv3d进行线性变换。NonlocalBlockND操作只需用到常用的卷积、矩阵相乘、加法、softmax等算子，用户可以非常方便的实现组网以构建模型。

2. nonlocal3d包含backbone、avg_pool、flatten、head几部分组成。大致可以归纳为如下几点。
第一部分：backbone部分是NLResInflate3D50（NLInflateResNet3D类），它是在NLInflateResNet3D结构中实现[3,4,6,3]规格的stage。而NLInflateResNet3D该结构是继承于ResNet3d50的结构，在ResNet3d50的[3,4,6,3]第2，3两个stage中的10层以每隔1层的方式插入一个NonlocalBlockND。
第二部分：NLResInflate3D50输出到一个平均池化并flatten,
第三部分：分类头。将flatten后的tensor输入到Dropdensehead进行分类，得到shape(N, NUM_CLASSES)的tensor。

In [7]:
from mindvideo.models.nonlocal3d import nonlocal3d
# Create model
network = nonlocal3d(in_d=32,
                     in_h=224,
                     in_w=224,
                     num_classes=400,
                     keep_prob=0.5)

In [9]:
from mindvideo.schedule.lr_schedule import warmup_step_lr
# Set learning rate scheduler.
lr = warmup_step_lr(lr=0.0003,
                    lr_epochs=[1],
                    steps_per_epoch=step_size,
                    warmup_epochs=1,
                    max_epoch=1,
                    gamma=0.1)

In [10]:
# Define optimizer.
network_opt = nn.SGD(network.trainable_params(),
                     lr,
                     momentum=0.9,
                     weight_decay=0.0001)
# Define loss function.
network_loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")

In [11]:
# set checkpoint for the network
ckpt_config = CheckpointConfig(
        save_checkpoint_steps=step_size,
        keep_checkpoint_max=1)
ckpt_callback = ModelCheckpoint(prefix='nonlocal_kinetics400',
                                directory=ckpt_save_dir,
                                config=ckpt_config)

In [15]:
# Init the model.
model = Model(network,
            loss_fn=network_loss,
            optimizer=network_opt,
            metrics={"Accuracy": Accuracy()})

In [17]:
# Begin to train.
print('[Start training `{}`]'.format('nonlocal_kinetics400'))
print("=" * 80)
model.train(1,
            dataset_train,
            callbacks=[ckpt_callback, LossMonitor()],
            dataset_sink_mode=False)
print('[End of training `{}`]'.format('nonlocal_kinetics400'))

[WARNING] ME(132676:140166743865152,MainProcess):2023-03-13-07:54:00.444.33 [mindspore/dataset/core/validator_helpers.py:804] 'Compose' from mindspore.dataset.transforms.py_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Compose' from mindspore.dataset.transforms instead.


[Start training `nonlocal_kinetics400`]
epoch: 1 step: 1, loss is 5.960698127746582
epoch: 1 step: 2, loss is 6.26731538772583
epoch: 1 step: 3, loss is 6.177163124084473


KeyboardInterrupt: 

### 评估流程

In [18]:
from mindspore import context
from mindspore.train.callback import Callback

class PrintEvalStep(Callback):
    """ print eval step """
    def step_end(self, run_context):
        """ eval step """
        cb_params = run_context.original_args()
        print("eval: {}/{}".format(cb_params.cur_step_num, cb_params.batch_num))

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")


In [19]:
from mindvideo.data.kinetics400 import Kinetic400

dataset_eval = Kinetic400(path="/home/publicfile/kinetics-400",
                          split="val",
                          shuffle=True,
                          seq=32,
                          seq_mode='interval',
                          num_parallel_workers=1,
                          batch_size=1,
                          frame_interval=6)

/home/publicfile/kinetics-400/cls2index.json


In [20]:
from mindvideo.data.transforms import VideoReOrder, VideoRescale, VideoNormalize
from mindvideo.data.transforms import VideoCenterCrop, VideoShortEdgeResize

transforms = [VideoShortEdgeResize(size=256, interpolation='bicubic'),
              VideoCenterCrop([224, 224]),
              VideoRescale(),
              VideoReOrder([3, 0, 1, 2]),
              VideoNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.255])]
dataset_eval.transform = transforms
dataset_eval = dataset_eval.run()

In [21]:
from mindspore import nn
from mindspore.train import Model
from mindspore.nn.loss import SoftmaxCrossEntropyWithLogits
from mindspore import load_checkpoint, load_param_into_net
from mindvideo.models.nonlocal3d import nonlocal3d

 # Create model.
network = nonlocal3d()


# Define loss function.
network_loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")

# Load pretrained model.
param_dict = load_checkpoint(ckpt_file_name='/home/hcx/nonlocal_mindspore/scripts/nonlocal_output_0.0003/nonlocal-1_4975.ckpt')
load_param_into_net(network, param_dict)

# Define eval_metrics.
eval_metrics = {'Loss': nn.Loss(),
                'Top_1_Accuracy': nn.Top1CategoricalAccuracy(),
                'Top_5_Accuracy': nn.Top5CategoricalAccuracy()}
print_cb = PrintEvalStep()

# Init the model.
model = Model(network, loss_fn=network_loss, metrics=eval_metrics)

In [22]:
# Begin to eval.
print('[Start eval `{}`]'.format('nonlocal_kinetics400'))
result = model.eval(dataset_eval,
                    callbacks=[print_cb],
                    dataset_sink_mode=False)
print(result)


[WARNING] ME(132676:140166743865152,MainProcess):2023-03-13-07:55:07.379.431 [mindspore/train/model.py:1077] For PrintEvalStep callback, {'step_end'} methods may not be supported in later version, Use methods prefixed with 'on_train' or 'on_eval' instead when using customized callbacks.
[WARNING] ME(132676:140166743865152,MainProcess):2023-03-13-07:55:07.381.857 [mindspore/dataset/core/validator_helpers.py:804] 'Compose' from mindspore.dataset.transforms.py_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Compose' from mindspore.dataset.transforms instead.
[WARNING] ME(132676:140166743865152,MainProcess):2023-03-13-07:55:07.394.380 [mindspore/dataset/core/validator_helpers.py:804] 'Compose' from mindspore.dataset.transforms.py_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Compose' from mindspore.dataset.transforms instead.
[WARNING] ME(132676:140166743865152,MainProcess):2023-03-13-07:55:07.398.446 [mindsp

[Start eval `nonlocal_kinetics400`]
eval: 1/19877
eval: 2/19877
eval: 3/19877
eval: 4/19877
eval: 5/19877
eval: 6/19877
eval: 7/19877
eval: 8/19877
eval: 9/19877
eval: 10/19877
eval: 11/19877
eval: 12/19877
eval: 13/19877
eval: 14/19877
eval: 15/19877
eval: 16/19877
eval: 17/19877
eval: 18/19877
eval: 19/19877
eval: 20/19877
eval: 21/19877
eval: 22/19877
eval: 23/19877


KeyboardInterrupt: 

## Code

[Gitee](https://gitee.com/yanlq46462828/zjut_mindvideo)

[GitHub](https://github.com/Aprilkaka/nonlocal_mindspore)
